# Airblast Mapper

In [1]:
import numpy as np
import pandas as pd
import armageddon

The Airblast Mapper Documentation can be divided into 3 sections:
- Damage Calculator


- Location / Postcode Finder


- Damage Mapper

# Damage Mapper Documentation

The Damage Mapper Functionality of the module was broken down into 3 python files. A damage solver file, a locator file and a mapper file.

## Damage Solver:

The initial damage solver file includes 2 main functions: 
- <u> Damage Zones</u> : which calculates the Latitude and Longitude of the zero point of the blast, as well as the blast radii depending on the total kinetic energy of the asteroid at the burst point and the pressures of the blast wave.


- <u> Impact Risk </u>:  which calculates the risk of each postcode/sector in the blast radii - with the uncertainty of the initial parameters of the asteroid.

## Damage Zones:

This function takes as parameters:
- Outcome: Is a Dictionary with the outcomes from an impact scenrario calculated from Airblast Solver.


- Latitude (lat): Is a float corresponding to the entry latitude of the asteroid. The unit of this parameter is degrees.


- Longitude (lon): Is a float corresponding to the entry longitude of the asteroid. The unit of this parameter is degrees.


- Bearing: Is a float corresponding to the Azimuth relativ to the north of the asteroid trajectory. The unit of this parameter is degrees.


- Pressures: Is as an (array - list) which includes all the thresholds deifning the airblast damage levels in Pa.

The Pressures used in our function were the following: 

|  Damage Level |  Description    | Pressure (kPa) |
|:-------------:|:---------------:|:--------------:|
|  1  |  ~10% glass windows shatter    |     1.0      |
|  2  | ~90% glass windows shatter     |     3.5      |
|  3  | Wood frame buildings collapse  |     27      |
|  4  | Multistory brick buildings collapse  |     43      |



The function ouputs:
- Blast Latitude (blat) as a float: The latitude of the surface zero point in degrees


- Blast Longitude (blon) as a float: The longitude of the surface zero point in degrees


- Damage radius (damrad) as an array of floats: The blast radii for the 4 input damage levels, in metres.

#### Example:

Using the following input Parameters:

- Outcome:
    - burst altitude: 8e3 metres
    - burst energy: 7e3 Kt-tnt
    - burst distance: 90e3 metres
    - burst peak dedz: 1e3
    - outcome: Airburst


- Entry Latitude: 52.79 degrees
- Entry Longitude: -2.95 degrees
- Bearing: 135 degrees
- Pressues: 1e3, 3.5e3, 27e3, 43e3


In [2]:
outcome = {'burst_altitude': 8e3, 'burst_energy': 7e3,
                   'burst_distance': 90e3, 'burst_peak_dedz': 1e3,
                   'outcome': 'Airburst'}
pressures=[1e3, 3.5e3, 27e3, 43e3]

blat, blon, damrad = armageddon.damage_zones(outcome, 52.79, -2.95, 135,
                     pressures)

print('Blast latitude was calculated to be at: ', blat, ' degrees.')
print()
print('Blast longitude was calculated to be at: ', blon, ' degrees.')
df = pd.DataFrame(data = (pressures, damrad), index=None).T
df.columns =  ['Pressures', 'Blast Radii (Metres)']
print()
print(df)

Blast latitude was calculated to be at:  52.21396905216969  degrees.

Blast longitude was calculated to be at:  -2.0159088616770733  degrees.

   Pressures  Blast Radii (Metres)
0     1000.0         115971.316730
1     3500.0          42628.366515
2    27000.0           9575.214234
3    43000.0           5835.983452


## Impact Risk

This function takes as inputs: 

- Planet Instance:  From the Solver Class


- The Fiducial means of the initial parameters of the asteroid fall as a <u>dictionary</u>:

    - Entry latitude 53.0$^\circ$
    - Entry longitude: -2.5$^\circ$
    - Entry bearing: 115.0$^\circ$
    - Meteoroid radius: 35 m
    - Meteoroid speed: 19 000 m/s
    - Meteoroid density: 3 000 kg/m$^3$
    - Meteoroid strength: 10 000 000 Pa\%
    - Meteoroid trajectory angle: 45$^\circ$
   

- The Fiducial standard deviations of the above parameters as a <u>dictionary</u>:  

    * Entry latitude 0.025$^\circ$
    * Entry longitude: 0.025$^\circ$
    * Entry bearing: 0.5$^\circ$
    * Meteoroid radius: 1 m
    * Meteoroid speed: 1000 m/s
    * Meteoroid density: 500 kg/m$^3$
    * Meteoroid strength: 50\%
    * Meteoroid trajectory angle: 1$^\circ$


 - A single pressure to calculate the impact zone as a float: 27 000 Pa


 - Number of Samples to take from the Gaussian Distributions


 - Sector: Boolean Input: 
     - True: Calculate the risk for postcode sectors
     - False: Calculate the risk for postcode units

The function outputs:

   - A pandas Dataframe containing the risk of the postcodes being affected by each of the sampled asteroid fallings.
   - The function should also save an HTML file where the map is plotted and can be used for emergency and evacuation situations.

### Example:

In [ ]:
planet = armageddon.Planet()
armageddon.impact_risk(planet)

# Location Documentation

The Location python file includes the great circle distance function and a get population class.

### Great Circle Distance Function:

This function calculates the distance between pairs of points (circle distance - as assumption that earth is speherical).

The function takes as inputs:
   
   - An array of size n x 2, with n pairs of latitudes and longitudes.
   - A second array of size m x 2 with m pairs of latitudes and longitudes.


The function ouputs:

- A n x m array containing the distances in metres between each pair of points.

### Example:

- Configure 2 lists of lists - as np-array size n x 2 and another of size m x 2.


- Apply the armageddon.great_circle_distance Function on those 2 arrays.

In [ ]:
pnts1 = np.array([[54.0, 0.0], [55.0, 1.0], [54.2, -3.0]])
pnts2 = np.array([[55.0, 1.0], [56.0, -2.1], [54.001, -0.003]])

data_ = armageddon.great_circle_distance(pnts1, pnts2)

df = pd.DataFrame(data = data_, columns=[str(pnts2[0]),
                                         str(pnts2[1]),
                                         str(pnts2[2])], 
                                  index=[str(pnts1[0]),
                                         str(pnts1[1]),
                                         str(pnts1[2])])
df

## Get Population Class: 

- Get Postcodes by radius Function: Function to find postcode sectors/units within a circle of radius radii and central location X.


- Get Population of postcodes Function:  Function that returns the population within a list of postcode sectors/units.


#### Assumptions: 

##### Missing Data:

   - It was noticed in our population by postcode sector dataset, that certain postcode sectors are not available.


   - As a way around printing an error, we have assumed that these areas have a population of 0, and are places such as industrial zones, large natural natural parks ect.. - This was implemented in the _add_0populations_postcode function.


   - We know this is not the most optimal solution, as some of these areas appear in Scotland and Ireland, but we have decided to continue with this.


### Get Postcodes by radius:

This function allows either all postcode sectors / units in a certain regions to be found.

This function takes as inputs:

- A pair of latitude-longitude (X): The central point of the blast as an array.

- Radii: The radial distances of X as an array in metres.


- Sector: A boolean parameter: When set to True, the function returns the postcode sectors found in the blast circle. When False returns the postcode units found in the blast circle.


The function then outputs:

- A list of lists containing the postcode sectors / units within the blast wave from the X location.

### Example:


- Create an instance of the class.


- Input the X, the central point of the blast as an array.


- Input the radii in metres.

In [ ]:
population_class = armageddon.PostcodeLocator()

Latitude_Longitude_pair = (51.4981, -0.1773)
Blast_radii = [0.13e3]

# Example which outputs the Postcode Units:
Postcode_unists = population_class.get_postcodes_by_radius(Latitude_Longitude_pair, 
                                         Blast_radii)
print('The postcode units located in the blast wave are the following: ', Postcode_unists)



# Example which outputs the Postcode Sectors:
Postcode_sectors = population_class.get_postcodes_by_radius(Latitude_Longitude_pair, 
                                         Blast_radii, True)
print('The postcode sectors located in the blast wave are the following: ', Postcode_sectors)

### Get Population of Postcodes:

This function allows the population of postcode sectors to be identified.


The function takes as input:


- Postcodes: as a list of lists of strings.


- Sector: A boolean parameter, when True returns the populations from a list of lists of postcode sectors. When False returns the populations from a list of lists of postcode units.


The function outputs: 

- A list of lists containing the population of input postcodes - rounded to the nearest full float number.

### Example: 

In [ ]:
# Example with list of lists of postcode units:

list_of_postcode_units = [['SW7 2AZ', 'SW7 2BT', 'SW7 2BU', 'SW7 2DD']]
print('The populations in the list of postcode units are: ', population_class.get_population_of_postcode(list_of_postcode_units))


# Example with list of lists of postcode sectors:

list_of_postcode_sectors = [['SW7 2', 'SW7 2']]
population_class.get_population_of_postcode(list_of_postcode_sectors, True)
print('The populations in the list of postcode sectors are: ', population_class.get_population_of_postcode(list_of_postcode_sectors, True))


# Mapper Documentation

The mapper python file allows the blast waves and population densities to be plotted on a map for easy usage in emergency response and evacuation planning. 

Large cities can be located close to the damage zones for smooth evacuation.


The file contains two functions : 

- Plot Circle
- Plot Impact Risk Circles


### Plot Circle:
Which takes as input the latitude and longitude of the blast centre, as well as the radius of the blast: 

- The latitude and longitude should be inputed as floats in degrees.
- The radius in metres as a list containing floats.
- The function also takes a map input which indicates if the function should take any existing map objects.


The function returns a Folium map, which can be printed or turned into a HTML file.

## Example

In [ ]:
# outcome = {'burst_altitude': 8e3, 'burst_energy': 7e3,
#            'burst_distance': 90e3, 'burst_peak_dedz': 1e3,
#            'outcome': 'Airburst'}
# blat, blon, damrad = armageddon.damage_zones(outcome, 52.79, -2.95, 135,
#                                   pressures=[1e3, 3.5e3, 27e3, 43e3])

# armageddon.plot_circle(blat, blon, damrad)

## Usage:

The plot_cirlce functions returns a map which is substantially zoomed out. For acurate representation of population density, you may be required to manually zoom onto a desired location.

In [ ]:
# import folium
# import armageddon
# armageddon.plot_circle(52.1951, -0.1313, [1e3, 3.5e3, 27e3, 43e3])